In [1]:
import pandas as pd
import geopandas as gpd
import re
from shapely.geometry import Polygon

In [2]:
df = pd.read_csv("taxable_values.csv",dtype=str)
df = df[0:37]
df['TAXING AUTHORITY'] = df['TAXING AUTHORITY'].str.replace('\d+','',regex=True)

In [3]:
gdf = gpd.read_file('Municipalitypoly_gdb_-3974701052066827405.geojson')

In [4]:
df['TAXING AUTHORITY'] = df['TAXING AUTHORITY'].str.strip()

In [5]:
gdf.columns

Index(['OBJECTID', 'MUNICUID', 'MUNICID', 'NAME', 'FIPSCODE', 'CREATEDBY',
       'CREATEDDATE', 'MODIFIEDBY', 'MODIFIEDDATE', 'GlobalID', 'geometry'],
      dtype='object')

In [6]:
merged_gdf = pd.merge(left=df,left_on='TAXING AUTHORITY',right=gdf,right_on='NAME',how='left')

In [7]:
merged_gdf['TAXING AUTHORITY'] = merged_gdf['TAXING AUTHORITY'].str.title()
merged_gdf['TAXING AUTHORITY'] = merged_gdf['TAXING AUTHORITY'].str.replace('Mb','MB')
merged_gdf['TAXING AUTHORITY'] = merged_gdf['TAXING AUTHORITY'].str.replace('Dda','DDA')

In [8]:
empty_poly = Polygon()
merged_gdf["geometry"] = merged_gdf["geometry"].apply(
    lambda geom: geom if geom is not None else empty_poly
)

In [9]:
def compute_centroid(geom):
    if geom is None or geom.is_empty:
        return None
    return geom.centroid

merged_gdf["centroid"] = merged_gdf["geometry"].apply(compute_centroid)

# 3) (Optional) If you want separate latitude/longitude columns:
merged_gdf["centroid_x"] = merged_gdf["centroid"].apply(lambda pt: pt.x if pt is not None else None)
merged_gdf["centroid_y"] = merged_gdf["centroid"].apply(lambda pt: pt.y if pt is not None else None)

In [10]:
drop_columns = ['OBJECTID', 'MUNICUID',
       'MUNICID', 'NAME', 'FIPSCODE', 'CREATEDBY', 'CREATEDDATE', 'MODIFIEDBY',
       'MODIFIEDDATE', 'GlobalID']

In [11]:
merged_gdf = merged_gdf.drop(columns=drop_columns)

In [13]:
merged_gdf = gpd.GeoDataFrame(
    merged_gdf,
    geometry="geometry",
    crs="EPSG:4326"    
)



In [14]:
merged_gdf

,TAXING AUTHORITY,2024 PRELIMINARY TAXABLE VALUE,2025 ESTIMATED TAXABLE VALUE BEFORE NEW CONSTRUCTION,PERCENT CHANGE,NEW CONSTRUCTION,2025 JUNE 1ST ESTIMATE,PERCENT CHANGE FROM 2024,NET VALUE CHANGE,geometry,centroid,centroid_x,centroid_y
0,Miami,94135310116,100330000000,6.6%,3070000000,103400000000,9.8%,9264000000,"POLYGON ((-80.16136 25.85594, -80.16079 25.843...",POINT (-80.20881969579519 25.775349375727767),-80.208820,25.775349
1,Miami ( DDA ),30067524964,30829000000,2.5%,1671000000,32500000000,8.1%,2432000000,POLYGON EMPTY,None,NaN,NaN
2,Miami Beach,56396947099,59374000000,5.3%,926000000,60300000000,6.9%,3903000000,"POLYGON ((-80.11962 25.87221, -80.11962 25.872...",POINT (-80.13952465878016 25.81693672367289),-80.139525,25.816937
3,Miami Beach,56396947099,59374000000,5.3%,926000000,60300000000,6.9%,3903000000,"POLYGON ((-80.13808 25.76297, -80.13808 25.762...",POINT (-80.14040338885435 25.761184903612254),-80.140403,25.761185
4,MB Normandy Shores,448663162,486000000,8.3%,1000000,487000000,8.5%,38000000,POLYGON EMPTY,None,NaN,NaN
5,Coral Gables,24572659959,25855000000,5.2%,45000000,25900000000,5.4%,1327000000,"POLYGON ((-80.25501 25.76112, -80.25469 25.752...",POINT (-80.25126036960482 25.67816052684459),-80.251260,25.678161
6,Hialeah,19972597788,21806000000,9.2%,294000000,22100000000,10.7%,2127000000,"POLYGON ((-80.34713 25.90026, -80.34675 25.899...",POINT (-80.30458808399048 25.869634653002308),-80.304588,25.869635
7,Miami Springs,1746158731,1872000000,7.2%,14000000,1886000000,8.0%,139000000,"POLYGON ((-80.28971 25.82929, -80.28971 25.829...",POINT (-80.28962980526056 25.819523972283715),-80.289630,25.819524
8,North Miami,5838979945,6199000000,6.2%,101000000,6300000000,7.9%,461000000,"POLYGON ((-80.13034 25.927, -80.13028 25.926, ...",POINT (-80.16844939413046 25.900757060586358),-80.168449,25.900757
9,North Miami,5838979945,6199000000,6.2%,101000000,6300000000,7.9%,461000000,"POLYGON ((-80.22746 25.89986, -80.22746 25.899...",POINT (-80.22649623815316 25.90061974503236),-80.226496,25.900620


In [16]:
merged_gdf.to_file("merged_taxable_values.geojson", driver="GeoJSON")

In [ ]:
import os

base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)